In [6]:
#importing the libraries needed 
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint

#Declaring the headers 
headers = {"Accept-Language": "en-US,en;q=0.5"}

#declaring the list of empty variables, So that we can append the data overall

movie_name = []
year = []
certificate=[]
time=[]
genres=[]
rating=[]
metascore =[]
votes = []
gross = []
description = []
Director = []
Stars = []

#creating an array of values and passing it in the url for dynamic webpages
pages = np.arange(1,10000,50)

#the whole core of the script
for page in pages:
    page = requests.get("https://www.imdb.com/search/title/?title_type=feature&num_votes=10000,&sort=user_rating,desc&start="+str(page)+"&ref_=adv_nxt%22")
    soup = BeautifulSoup(page.text, 'html.parser')
    movie_data = soup.findAll('div', attrs = {'class': 'lister-item mode-advanced'})
    sleep(randint(2,8))
    for store in movie_data:
        name = store.h3.a.text
        movie_name.append(name)
        
        year_of_release = store.h3.find('span', class_ = "lister-item-year text-muted unbold").text
        year.append(year_of_release)
        
        
        certificates = store.p.find("span", class_ = 'certificate').text if store.find('span', class_ = "certificate") else "****"
        certificate.append(certificates)
        
        
        runtime = store.p.find("span", class_ = 'runtime').text
        time.append(runtime)
        
        genre = store.p.find('span', class_ = 'genre').text.rstrip().replace("\n","").split(',')
        genres.append(genre)
        
        rate = store.find('div', class_ = "inline-block ratings-imdb-rating").text.replace('\n', '')
        rating.append(rate)
        
        meta = store.find('span', class_ = "metascore").text if store.find('span', class_ = "metascore") else "****"
        metascore.append(meta)
        
        
        value = store.find_all('span', attrs = {'name': "nv"})
        
        vote = value[0].text
        votes.append(vote)
        
        grosses = value[1].text if len(value)>1 else '%^%^%^'
        gross.append(grosses)
        
        # Description of the Movies -- Not explained in the Video, But you will figure it out. 
        describe = store.find_all('p', class_ = 'text-muted')
        description_ = describe[1].text.replace('\n', '') if len(describe) >1 else '*****'
        description.append(description_)
        
        
        Cast = store.find('p', class_='')
        casts = Cast.text.replace("\n","").split('|')
        casts=[x.strip() for x in casts] if len(casts)>1 else '###'
        casts=[casts[i].replace(j,'') for i,j in enumerate(["Director:","Stars:"])]
        Director.append(casts[0])
        Stars.append([x.strip() for x in casts[1].split(',')])
        
    
    
#creating a dataframe using pandas library
movie_list = pd.DataFrame({'Name of movie': movie_name, 'Year of relase': year,'Certificate': certificate, 'Watchtime': time,'movie genre':genres, 'Movie Rating': rating, 'Metascore': metascore, 'Votes': votes, 'Gross collection': gross, 'Description': description, "Director": Director, 'Star': Stars})


In [7]:
movie_list.head(5)

,Name of movie,Year of relase,Certificate,Watchtime,movie genre,Movie Rating,Metascore,Votes,Gross collection,Description,Director,Star
0,The Shawshank Redemption,(1994),A,142 min,[Drama],9.3,82,"2,808,505",$28.34M,"Over the course of several years, two convicts...",Frank Darabont,"[Tim Robbins, Morgan Freeman, Bob Gunton, Will..."
1,The Godfather,(1972),A,175 min,"[Crime, Drama]",9.2,100,"1,957,175",$134.97M,"Don Vito Corleone, head of a mafia family, dec...",Francis Ford Coppola,"[Marlon Brando, Al Pacino, James Caan, Diane K..."
2,Ramayana: The Legend of Prince Rama,(1993),U,135 min,"[Animation, Action, Adventure]",9.2,****,"13,054",%^%^%^,An anime adaptation of the Hindu epic the Rama...,"Directors:Ram Mohan, Yûgô Sakô, Koichi Saski","[Arun Govil, Nikhil Kapoor, Edie Mirman, Rael ..."
3,Hababam Sinifi,(1975),****,87 min,"[Comedy, Drama]",9.2,****,"42,258",%^%^%^,"Lazy, uneducated students share a very close b...",Ertem Egilmez,"[Kemal Sunal, Münir Özkul, Halit Akçatepe, Tar..."
4,The Dark Knight,(2008),UA,152 min,"[Action, Crime, Drama]",9.0,84,"2,790,224",$534.86M,When the menace known as the Joker wreaks havo...,Christopher Nolan,"[Christian Bale, Heath Ledger, Aaron Eckhart, ..."


In [8]:
# #saving the data in excel format
movie_list.to_excel("Top 10000 IMDb movies.xlsx")

# #If you want to save the data in csv format
movie_list.to_csv("Top 10000 IMDb movies.csv")

In [9]:
movie_list.shape

(10000, 12)